In [ ]:
import datetime
import sqlite3
import urllib.parse

import requests

from config import weather_api_key  # OpenWeatherMap API key from config.py

In [72]:
CRASH_DATA_URL = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?$query=SELECT%0A%20%20%60crash_date%60%2C%0A%20%20%60crash_time%60%2C%0A%20%20%60borough%60%2C%0A%20%20%60zip_code%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%2C%0A%20%20%60on_street_name%60%2C%0A%20%20%60off_street_name%60%2C%0A%20%20%60cross_street_name%60%2C%0A%20%20%60number_of_persons_injured%60%2C%0A%20%20%60number_of_persons_killed%60%2C%0A%20%20%60number_of_pedestrians_injured%60%2C%0A%20%20%60number_of_pedestrians_killed%60%2C%0A%20%20%60number_of_cyclist_injured%60%2C%0A%20%20%60number_of_cyclist_killed%60%2C%0A%20%20%60number_of_motorist_injured%60%2C%0A%20%20%60number_of_motorist_killed%60%2C%0A%20%20%60contributing_factor_vehicle_1%60%2C%0A%20%20%60contributing_factor_vehicle_2%60%2C%0A%20%20%60contributing_factor_vehicle_3%60%2C%0A%20%20%60contributing_factor_vehicle_4%60%2C%0A%20%20%60contributing_factor_vehicle_5%60%2C%0A%20%20%60collision_id%60%2C%0A%20%20%60vehicle_type_code1%60%2C%0A%20%20%60vehicle_type_code2%60%2C%0A%20%20%60vehicle_type_code_3%60%2C%0A%20%20%60vehicle_type_code_4%60%2C%0A%20%20%60vehicle_type_code_5%60%0AWHERE%0A%20%20%60crash_date%60%0A%20%20%20%20BETWEEN%20%222022-01-01T17%3A20%3A02%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222024-12-03T17%3A20%3A02%22%20%3A%3A%20floating_timestamp%0AORDER%20BY%20%60crash_date%60%20ASC%20NULL%20LAST"
WEATHER_DATA_URL = "https://openweathermap.org/history"
WEATHER_API_APPID = weather_api_key
SQLITE_DB = "resources/weather_crash_data.db"

In [73]:
def extract():
    try:
        crash_data = extract_crash_data()
    except:
        crash_data = "ERROR"
    try:
        crash_locations = get_crash_locations(crash_data)
        weather_data = extract_weather_data(crash_locations)
    except:
        weather_data = "ERROR"
    
    return dict(
        crash_data=crash_data,
        weather_data=weather_data,
    )

In [74]:
def extract_crash_data():
    crash_data_url = CRASH_DATA_URL
    response = requests.get(crash_data_url)
    return response.json()

In [75]:
def get_crash_locations(crash_data):
    locations = set()
    for record in crash_data:
        try:
            lat, lon = record["latitude"], record["longitude"]
            assert float(lat) and float(lon)
            location = (lat, lon)
            locations.add(location)
        except KeyError:
            pass
        except AssertionError:
            pass
    return sorted(locations)

In [76]:
def extract_weather_data(locations):
    assert locations and isinstance(locations, list)
    
    weather_data = {}
    for location in locations:
        lat, lon = location
        type_ = "hour"
        appid = WEATHER_API_APPID
        start = datetime.date(2022, 1, 1).toordinal()
        weather_data[location] = extract_weather_data_for_location(location)
    return response.json()

In [77]:
def extract_weather_data_for_location(location):
    lat, lon = location
    # weather_data_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={appid}"
    weather_data_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&appid={appid}"
    response = requests.get(weather_data_url)
    return response.json()

In [78]:
def est_date_to_utc_timestamp(est_date):
    """Convert a date in EST timezone to a UTC timestamp."""
    est_timestamp = est_date.timestamp()
    utc_timestamp = est_timestamp + 5 * 3600
    return utc_timestamp

In [79]:
# Prototyping crash data extraction and processing
# crash_data = extract_crash_data()
# crash_locations = get_crash_locations(crash_data)

# center_lat, center_lon, n = 0.0, 0.0, len(crash_locations)
# for location in crash_locations:
#     lat, lon = location
#     center_lat += float(lat)
#     center_lon += float(lon)
# center_lat /= n
# center_lon /= n
# (center_lat, center_lon)
# manhattan = (center_lat, center_lon)

In [80]:
# Extract weather data for one location in Manhattan
# weather_data = extract_weather_data_for_location(manhattan)

In [ ]:
# Prototyping getting weather data for one location in Manhattan

# Constant timestamps for prototyping
jan_1_2022_timestamp = datetime.datetime(2022, 1, 1, 0, 0, 0).timestamp()
jan_1_2025_timestamp = datetime.datetime(2025, 1, 1, 0, 0, 0).timestamp()
dec_3_2024_timestamp = datetime.datetime(2024, 12, 3, 23, 59, 59).timestamp()

# manhattan
# start, end = jan_1_2022_timestamp, dec_3_2024_timestamp
# lat, lon = manhattan
# appid = "REDACTED"
# url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={appid}"
# response = requests.get(url)
# if response.status_code != 200:
#     print(response.status_code)
#     print(response.content)
# else:
#     weather_data = response.json()
#     print(weather_data)

In [12]:
crash_data = extract_crash_data()
# weather_crash_data = extract()

In [13]:
crash_data[0]

{'crash_date': '2022-01-02T00:00:00.000',
 'crash_time': '9:20',
 'on_street_name': 'VANDAM STREET',
 'off_street_name': 'BORDEN AVENUE',
 'number_of_persons_injured': '0',
 'number_of_persons_killed': '0',
 'number_of_pedestrians_injured': '0',
 'number_of_pedestrians_killed': '0',
 'number_of_cyclist_injured': '0',
 'number_of_cyclist_killed': '0',
 'number_of_motorist_injured': '0',
 'number_of_motorist_killed': '0',
 'contributing_factor_vehicle_1': 'Failure to Yield Right-of-Way',
 'contributing_factor_vehicle_2': 'Unspecified',
 'collision_id': '4491517',
 'vehicle_type_code1': 'Sedan',
 'vehicle_type_code2': 'Box Truck'}

In [14]:
data = extract()

In [15]:
data["crash_data"][0]

{'crash_date': '2022-01-02T00:00:00.000',
 'crash_time': '9:20',
 'on_street_name': 'VANDAM STREET',
 'off_street_name': 'BORDEN AVENUE',
 'number_of_persons_injured': '0',
 'number_of_persons_killed': '0',
 'number_of_pedestrians_injured': '0',
 'number_of_pedestrians_killed': '0',
 'number_of_cyclist_injured': '0',
 'number_of_cyclist_killed': '0',
 'number_of_motorist_injured': '0',
 'number_of_motorist_killed': '0',
 'contributing_factor_vehicle_1': 'Failure to Yield Right-of-Way',
 'contributing_factor_vehicle_2': 'Unspecified',
 'collision_id': '4491517',
 'vehicle_type_code1': 'Sedan',
 'vehicle_type_code2': 'Box Truck'}

In [21]:
# Data loading SQL queries
CREATE_CRASH_DATA_TABLE_SQL = """
    CREATE TABLE IF NOT EXISTS crash_data (
        crash_date TEXT,
        crash_time TEXT,
        borough TEXT,
        zip_code TEXT,
        latitude TEXT,
        longitude TEXT,
        location TEXT,
        on_street_name TEXT,
        off_street_name TEXT,
        cross_street_name TEXT,
        number_of_persons_injured INTEGER,
        number_of_persons_killed INTEGER,
        number_of_pedestrians_injured INTEGER,
        number_of_pedestrians_killed INTEGER,
        number_of_cyclist_injured INTEGER,
        number_of_cyclist_killed INTEGER,
        number_of_motorist_injured INTEGER,
        number_of_motorist_killed INTEGER,
        contributing_factor_vehicle_1 TEXT,
        contributing_factor_vehicle_2 TEXT,
        contributing_factor_vehicle_3 TEXT,
        contributing_factor_vehicle_4 TEXT,
        contributing_factor_vehicle_5 TEXT,
        collision_id TEXT,
        vehicle_type_code1 TEXT,
        vehicle_type_code2 TEXT,
        vehicle_type_code_3 TEXT,
        vehicle_type_code_4 TEXT,
        vehicle_type_code_5 TEXT
    )
"""
INSERT_CRASH_DATA_SQL = """
    INSERT INTO crash_data (
        crash_date,
        crash_time,
        borough,
        zip_code,
        latitude,
        longitude,
        location,
        on_street_name,
        off_street_name,
        cross_street_name,
        number_of_persons_injured,
        number_of_persons_killed,
        number_of_pedestrians_injured,
        number_of_pedestrians_killed,
        number_of_cyclist_injured,
        number_of_cyclist_killed,
        number_of_motorist_injured,
        number_of_motorist_killed,
        contributing_factor_vehicle_1,
        contributing_factor_vehicle_2,
        contributing_factor_vehicle_3,
        contributing_factor_vehicle_4,
        contributing_factor_vehicle_5,
        collision_id,
        vehicle_type_code1,
        vehicle_type_code2,
        vehicle_type_code_3,
        vehicle_type_code_4,
        vehicle_type_code_5
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

In [ ]:
# Data loading SQL queries
CREATE_CRASH_DATA_TABLE_SQL = """
    CREATE TABLE IF NOT EXISTS crash_data (
        crash_date TEXT,
        crash_time TEXT,
        borough TEXT,
        zip_code TEXT,
        latitude TEXT,
        longitude TEXT,
        location TEXT,
        on_street_name TEXT,
        off_street_name TEXT,
        cross_street_name TEXT,
        number_of_persons_injured INTEGER,
        number_of_persons_killed INTEGER,
        number_of_pedestrians_injured INTEGER,
        number_of_pedestrians_killed INTEGER,
        number_of_cyclist_injured INTEGER,
        number_of_cyclist_killed INTEGER,
        number_of_motorist_injured INTEGER,
        number_of_motorist_killed INTEGER,
        contributing_factor_vehicle_1 TEXT,
        contributing_factor_vehicle_2 TEXT,
        contributing_factor_vehicle_3 TEXT,
        contributing_factor_vehicle_4 TEXT,
        contributing_factor_vehicle_5 TEXT,
        collision_id TEXT,
        vehicle_type_code1 TEXT,
        vehicle_type_code2 TEXT,
        vehicle_type_code_3 TEXT,
        vehicle_type_code_4 TEXT,
        vehicle_type_code_5 TEXT
    )
"""
INSERT_CRASH_DATA_SQL = """
    INSERT INTO crash_data (
        crash_date,
        crash_time,
        borough,
        zip_code,
        latitude,
        longitude,
        location,
        on_street_name,
        off_street_name,
        cross_street_name,
        number_of_persons_injured,
        number_of_persons_killed,
        number_of_pedestrians_injured,
        number_of_pedestrians_killed,
        number_of_cyclist_injured,
        number_of_cyclist_killed,
        number_of_motorist_injured,
        number_of_motorist_killed,
        contributing_factor_vehicle_1,
        contributing_factor_vehicle_2,
        contributing_factor_vehicle_3,
        contributing_factor_vehicle_4,
        contributing_factor_vehicle_5,
        collision_id,
        vehicle_type_code1,
        vehicle_type_code2,
        vehicle_type_code_3,
        vehicle_type_code_4,
        vehicle_type_code_5
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

In [66]:
# Load the data into the SQLite database
def load(data):
    db = get_sqlite_db()
    crash_data = data["crash_data"]
    # weather_data = data["weather_data"]
    load_crash_data(db, crash_data)
    # load_weather_data(db, weather_data)
    


def get_sqlite_db():
    conn = sqlite3.connect(SQLITE_DB)
    return conn


def load_crash_data(db, crash_data):
    print(f"There are {len(crash_data)} records to load.")
    
    cursor = db.cursor()
    cursor.execute(CREATE_CRASH_DATA_TABLE_SQL)
    # cursor.executemany(INSERT_CRASH_DATA_SQL, crash_data) # May have errors
    # for i, record in enumerate(crash_data, start=1):
    for i, record in enumerate(crash_data[:10], start=1):
        clean_record = clean_crash_record(record)
        # Check the clean_record
        assert len(clean_record) == 29
        try:
            values = clean_record.values()
            cursor.execute(INSERT_CRASH_DATA_SQL, values)
            cursor.commit()
            db.commit()
            print(f"Inserted record {i}.")
        except Exception as e:
            print(f"Error inserting record {i}: {values}.")
            print(e)
            pass


def clean_crash_record(record):
    clean_record = dict(
        crash_date=record.get("crash_date", None),
        crash_time=record.get("crash_time", None),
        borough=record.get("borough", None),
        zip_code=record.get("zip_code", None),
        latitude=record.get("latitude", None),
        longitude=record.get("longitude", None),
        # location=record.get("location", None),
        location=None, # Nester and redundant, already have lat/lon fields
        on_street_name=record.get("on_street_name", None),
        off_street_name=record.get("off_street_name", None),
        cross_street_name=record.get("cross_street_name", None),
        number_of_persons_injured=record.get("number_of_persons_injured", None),
        number_of_persons_killed=record.get("number_of_persons_killed", None),
        number_of_pedestrians_injured=record.get("number_of_pedestrians_injured", None),
        number_of_pedestrians_killed=record.get("number_of_pedestrians_killed", None),
        number_of_cyclist_injured=record.get("number_of_cyclist_injured", None),
        number_of_cyclist_killed=record.get("number_of_cyclist_killed", None),
        number_of_motorist_injured=record.get("number_of_motorist_injured", None),
        number_of_motorist_killed=record.get("number_of_motorist_killed", None),
        contributing_factor_vehicle_1=record.get("contributing_factor_vehicle_1", None),
        contributing_factor_vehicle_2=record.get("contributing_factor_vehicle_2", None),
        contributing_factor_vehicle_3=record.get("contributing_factor_vehicle_3", None),
        contributing_factor_vehicle_4=record.get("contributing_factor_vehicle_4", None),
        contributing_factor_vehicle_5=record.get("contributing_factor_vehicle_5", None),
        collision_id=record.get("collision_id", None),
        vehicle_type_code1=record.get("vehicle_type_code1", None),
        vehicle_type_code2=record.get("vehicle_type_code2", None),
        vehicle_type_code_3=record.get("vehicle_type_code_3", None),
        vehicle_type_code_4=record.get("vehicle_type_code_4", None),
        vehicle_type_code_5=record.get("vehicle_type_code_5", None),
    )
    return clean_record
import csv
def load_to_csv(data, csv_filename):
    with open(f'resources/{csv_filename}', 'w', encoding='utf-8', newline='\n') as f:
        clean_data = [clean_crash_record(record) for record in data]
        writer = csv.DictWriter(f, fieldnames=clean_data[0].keys())
        writer.writeheader()
        writer.writerows(clean_data)


In [67]:
load_to_csv(data["crash_data"], "crash_data.csv")

In [62]:
load(data)

There are 1000 records to load.
Error inserting record 1: dict_values(['2022-01-02T00:00:00.000', '9:20', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'VANDAM STREET', 'BORDEN AVENUE', 'NULL', '0', '0', '0', '0', '0', '0', '0', '0', 'Failure to Yield Right-of-Way', 'Unspecified', 'NULL', 'NULL', 'NULL', '4491517', 'Sedan', 'Box Truck', 'NULL', 'NULL', 'NULL']).
parameters are of unsupported type
Error inserting record 2: dict_values(['2022-01-02T00:00:00.000', '12:50', 'QUEENS', '11373', '40.735065', '-73.88184', 'NULL', 'GRAND AVENUE', 'VANKLEECK STREET', 'NULL', '0', '0', '0', '0', '0', '0', '0', '0', 'Unspecified', 'Unspecified', 'NULL', 'NULL', 'NULL', '4492016', 'Sedan', 'Bike', 'NULL', 'NULL', 'NULL']).
parameters are of unsupported type
Error inserting record 3: dict_values(['2022-01-02T00:00:00.000', '0:30', 'NULL', 'NULL', '40.69919', '-73.91469', 'NULL', 'IRVING AVENUE', 'NULL', 'NULL', '0', '0', '0', '0', '0', '0', '0', '0', 'Turning Improperly', 'NULL', 'NULL', 'NULL', 'NULL', '

In [ ]:
lat, lon = 40.7128, -74.0060
appid = "REDACTED"
response = requests.get(f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&appid={appid}")
print(f"Status Code: {response.status_code}")
print(f"Content: {response.content}")

Status Code: 401
Content: b'{"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}'
